Initial setup:
```
source /global/cfs/cdirs/desi/software/desi_environment.sh 20.12
module load parallel

export INDIR=/global/cfs/cdirs/desi/spectro/redux/blanc/exposures
export OUTDIR=/global/cfs/cdirs/desi/users/$USER/redux/blanc/single_exposures
```

Run desi_group_spectra:
```
salloc -N 1 -C haswell -q interactive -t 4:00:00
parallel --jobs 32 < desi_group_spectra.txt ; exit
```

Run redrock:
```
salloc -N 50 --qos interactive -C haswell -t 4:00:00
parallel --jobs 50 --delay 1 < rrdesi.txt ; exit
```

In [1]:
from __future__ import division, print_function
import sys, os, glob, time, warnings, gc
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, vstack, hstack
import fitsio
# from astropy.io import fits

In [2]:
params = {'legend.fontsize': 'large',
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large',
         'figure.facecolor':'w'} 
plt.rcParams.update(params)

In [3]:
n_exp = 1
overwrite = False

# indir = os.getenv('INDIR')
# output_dir = os.getenv('OUTDIR')
indir = '/global/cfs/cdirs/desi/spectro/redux/blanc/exposures'
outdir = os.path.expandvars('/global/cfs/cdirs/desi/users/$USER/redux/blanc/single_exposures')
os.environ['INDIR'] = indir
os.environ['OUTDIR'] = outdir

exposure_list = Table.read('/global/cfs/cdirs/desi/spectro/redux/blanc/run/scripts/tile/allexp_sort.dat', format='ascii')
exposure_list.rename_columns(('col1', 'col2', 'col3'), ('tileid', 'date', 'expid'))
expid_exposure_list = list(exposure_list['expid'])
print(len(expid_exposure_list))

450


In [4]:
cframes_stack = []

for index in range(len(exposure_list)):
    tileid = exposure_list['tileid'][index]
    date = exposure_list['date'][index]
    expid = exposure_list['expid'][index]
    exposure_dir = os.path.join(indir, str(date), str(expid).zfill(8))
    cframe_list = glob.glob(os.path.join(exposure_dir, 'cframe-*'))
    if len(cframe_list)==0:
        print(exposure_dir, 'is empty')
        continue

    cframes = Table()
    cframes['cframe_fn'] = np.array(cframe_list)
    cframes['tileid'] = np.zeros(len(cframes), dtype=int)
    cframes['expid'] = np.zeros(len(cframes), dtype=int)
    cframes['camera'] = ' '
    cframes['petal_loc'] = -1 * np.ones(len(cframes), dtype=np.int32)

    for cframe_index, cframe_fn in enumerate(cframe_list):
        _, cp, expid1 = os.path.basename(cframe_fn).strip('.fits').split('-')
        camera, petal_loc = cp[0], cp[1]
        expid1 = int(expid)
        if expid1!=expid:
            raise ValueError()
        cframes['tileid'][cframe_index] = tileid
        cframes['expid'][cframe_index] = expid
        cframes['camera'][cframe_index] = camera
        cframes['petal_loc'][cframe_index] = petal_loc
    
    cframes_stack.append(cframes)
    
cframes = vstack(cframes_stack)
print(len(cframes))
print(len(np.unique(cframes['expid'])))

cframes[:1]

/global/cfs/cdirs/desi/spectro/redux/blanc/exposures/20200227/00052446 is empty
/global/cfs/cdirs/desi/spectro/redux/blanc/exposures/20200229/00052790 is empty
11559
448


cframe_fn,tileid,expid,camera,petal_loc
str94,int64,int64,str1,int32
/global/cfs/cdirs/desi/spectro/redux/blanc/exposures/20200314/00055456/cframe-r7-00055456.fits,65008,55456,r,7


In [5]:
cframes['keep'] = False
for expid in np.unique(cframes['expid']):
    mask = cframes['expid']==expid
    for petal_loc in np.arange(10):
        mask1 = mask & (cframes['petal_loc']==petal_loc)
        if np.sum(mask1)==3:  # require all 3 cameras
            cframes['keep'][mask1] = True
print(np.sum(cframes['keep']), np.sum(~cframes['keep']))

cframes = cframes[cframes['keep']]
print(len(cframes))

11559 0
11559


In [6]:
# mask = cframes['tileid']==65008
# cframes = cframes[mask]

In [7]:
# keep only one camera for simplicity
mask = cframes['camera']=='b'
cframes = cframes[mask]
print(len(cframes))

cframes.sort('cframe_fn')

3853


## Print desi_group_spectra commands

In [8]:
command_output_path_all = 'desi_group_spectra_all.txt'
command_output_path = 'desi_group_spectra.txt'  # commands for reruns (e.g. after failures)

f_all = open(command_output_path_all, 'w')
f = open(command_output_path, 'w')

for expid in np.unique(cframes['expid']):

    mask = (cframes['expid']==expid)
    tileid = cframes['tileid'][mask][0]
    
    for petal_loc in np.unique(cframes['petal_loc'][mask]):
        
        exposure_dir = os.path.dirname(cframes['cframe_fn'][mask][0])
        input_loc = os.path.join(exposure_dir.replace(indir, '$INDIR'), 'cframe-[brz]{}-*.fits'.format(petal_loc))

        # Stephen's suggested directory and filename format
        expid_str = str(expid).zfill(8)
        spectra_output_loc = os.path.join('$OUTDIR', str(tileid), 'spectra-{}-{}-{}.fits'.format(petal_loc, tileid, expid_str))

        f_all.write('desi_group_spectra --inframes {} --outfile {}\n'.format(input_loc, spectra_output_loc))
        # print('desi_group_spectra --inframes {} --outfile {}'.format(input_argument, output_argument))
        
        if os.path.isfile(os.path.expandvars(spectra_output_loc)) and (not overwrite):
            print('Warninig: {} already exists!'.format(spectra_output_loc))
        else:
            f.write('desi_group_spectra --inframes {} --outfile {}\n'.format(input_loc, spectra_output_loc))
f.close()
f_all.close()

Warninig: $OUTDIR/70004/spectra-0-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/spectra-2-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/spectra-3-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/spectra-6-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/spectra-7-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/spectra-9-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/spectra-0-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/spectra-2-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/spectra-3-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/spectra-6-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/spectra-7-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/spectra-9-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/spectra-0-70004-00051001.fits already exists!
Warninig: $OUTDIR/70004/spectra-2-70004-00051001.fits already exists!
Warninig: $OUTDIR/70

Warninig: $OUTDIR/70003/spectra-0-70003-00052028.fits already exists!
Warninig: $OUTDIR/70003/spectra-2-70003-00052028.fits already exists!
Warninig: $OUTDIR/70003/spectra-3-70003-00052028.fits already exists!
Warninig: $OUTDIR/70003/spectra-6-70003-00052028.fits already exists!
Warninig: $OUTDIR/70003/spectra-7-70003-00052028.fits already exists!
Warninig: $OUTDIR/70003/spectra-9-70003-00052028.fits already exists!
Warninig: $OUTDIR/70003/spectra-0-70003-00052030.fits already exists!
Warninig: $OUTDIR/70003/spectra-2-70003-00052030.fits already exists!
Warninig: $OUTDIR/70003/spectra-3-70003-00052030.fits already exists!
Warninig: $OUTDIR/70003/spectra-6-70003-00052030.fits already exists!
Warninig: $OUTDIR/70003/spectra-7-70003-00052030.fits already exists!
Warninig: $OUTDIR/70003/spectra-9-70003-00052030.fits already exists!
Warninig: $OUTDIR/70003/spectra-0-70003-00052032.fits already exists!
Warninig: $OUTDIR/70003/spectra-2-70003-00052032.fits already exists!
Warninig: $OUTDIR/70

Warninig: $OUTDIR/70512/spectra-2-70512-00052309.fits already exists!
Warninig: $OUTDIR/70512/spectra-3-70512-00052309.fits already exists!
Warninig: $OUTDIR/70512/spectra-6-70512-00052309.fits already exists!
Warninig: $OUTDIR/70512/spectra-7-70512-00052309.fits already exists!
Warninig: $OUTDIR/70512/spectra-9-70512-00052309.fits already exists!
Warninig: $OUTDIR/70512/spectra-0-70512-00052310.fits already exists!
Warninig: $OUTDIR/70512/spectra-2-70512-00052310.fits already exists!
Warninig: $OUTDIR/70512/spectra-3-70512-00052310.fits already exists!
Warninig: $OUTDIR/70512/spectra-6-70512-00052310.fits already exists!
Warninig: $OUTDIR/70512/spectra-7-70512-00052310.fits already exists!
Warninig: $OUTDIR/70512/spectra-9-70512-00052310.fits already exists!
Warninig: $OUTDIR/70003/spectra-0-70003-00052323.fits already exists!
Warninig: $OUTDIR/70003/spectra-2-70003-00052323.fits already exists!
Warninig: $OUTDIR/70003/spectra-3-70003-00052323.fits already exists!
Warninig: $OUTDIR/70

Warninig: $OUTDIR/70500/spectra-6-70500-00052937.fits already exists!
Warninig: $OUTDIR/70500/spectra-7-70500-00052937.fits already exists!
Warninig: $OUTDIR/70500/spectra-9-70500-00052937.fits already exists!
Warninig: $OUTDIR/70500/spectra-0-70500-00052938.fits already exists!
Warninig: $OUTDIR/70500/spectra-3-70500-00052938.fits already exists!
Warninig: $OUTDIR/70500/spectra-6-70500-00052938.fits already exists!
Warninig: $OUTDIR/70500/spectra-7-70500-00052938.fits already exists!
Warninig: $OUTDIR/70500/spectra-9-70500-00052938.fits already exists!
Warninig: $OUTDIR/70500/spectra-0-70500-00052939.fits already exists!
Warninig: $OUTDIR/70500/spectra-3-70500-00052939.fits already exists!
Warninig: $OUTDIR/70500/spectra-6-70500-00052939.fits already exists!
Warninig: $OUTDIR/70500/spectra-7-70500-00052939.fits already exists!
Warninig: $OUTDIR/70500/spectra-9-70500-00052939.fits already exists!
Warninig: $OUTDIR/70500/spectra-0-70500-00052940.fits already exists!
Warninig: $OUTDIR/70

Warninig: $OUTDIR/70006/spectra-0-70006-00053129.fits already exists!
Warninig: $OUTDIR/70006/spectra-3-70006-00053129.fits already exists!
Warninig: $OUTDIR/70006/spectra-6-70006-00053129.fits already exists!
Warninig: $OUTDIR/70006/spectra-7-70006-00053129.fits already exists!
Warninig: $OUTDIR/70006/spectra-9-70006-00053129.fits already exists!
Warninig: $OUTDIR/70006/spectra-0-70006-00053521.fits already exists!
Warninig: $OUTDIR/70006/spectra-1-70006-00053521.fits already exists!
Warninig: $OUTDIR/70006/spectra-3-70006-00053521.fits already exists!
Warninig: $OUTDIR/70006/spectra-5-70006-00053521.fits already exists!
Warninig: $OUTDIR/70006/spectra-6-70006-00053521.fits already exists!
Warninig: $OUTDIR/70006/spectra-7-70006-00053521.fits already exists!
Warninig: $OUTDIR/70006/spectra-8-70006-00053521.fits already exists!
Warninig: $OUTDIR/70006/spectra-9-70006-00053521.fits already exists!
Warninig: $OUTDIR/70006/spectra-0-70006-00053524.fits already exists!
Warninig: $OUTDIR/70

...

Warninig: $OUTDIR/80612/spectra-9-80612-00069038.fits already exists!
Warninig: $OUTDIR/80612/spectra-0-80612-00069039.fits already exists!
Warninig: $OUTDIR/80612/spectra-1-80612-00069039.fits already exists!
Warninig: $OUTDIR/80612/spectra-2-80612-00069039.fits already exists!
Warninig: $OUTDIR/80612/spectra-3-80612-00069039.fits already exists!
Warninig: $OUTDIR/80612/spectra-4-80612-00069039.fits already exists!
Warninig: $OUTDIR/80612/spectra-5-80612-00069039.fits already exists!
Warninig: $OUTDIR/80612/spectra-6-80612-00069039.fits already exists!
Warninig: $OUTDIR/80612/spectra-7-80612-00069039.fits already exists!
Warninig: $OUTDIR/80612/spectra-8-80612-00069039.fits already exists!
Warninig: $OUTDIR/80612/spectra-9-80612-00069039.fits already exists!
Warninig: $OUTDIR/80612/spectra-0-80612-00069040.fits already exists!
Warninig: $OUTDIR/80612/spectra-1-80612-00069040.fits already exists!
Warninig: $OUTDIR/80612/spectra-2-80612-00069040.fits already exists!
Warninig: $OUTDIR/80

Warninig: $OUTDIR/80619/spectra-6-80619-00069244.fits already exists!
Warninig: $OUTDIR/80619/spectra-7-80619-00069244.fits already exists!
Warninig: $OUTDIR/80619/spectra-8-80619-00069244.fits already exists!
Warninig: $OUTDIR/80619/spectra-9-80619-00069244.fits already exists!
Warninig: $OUTDIR/80619/spectra-0-80619-00069245.fits already exists!
Warninig: $OUTDIR/80619/spectra-1-80619-00069245.fits already exists!
Warninig: $OUTDIR/80619/spectra-2-80619-00069245.fits already exists!
Warninig: $OUTDIR/80619/spectra-3-80619-00069245.fits already exists!
Warninig: $OUTDIR/80619/spectra-4-80619-00069245.fits already exists!
Warninig: $OUTDIR/80619/spectra-5-80619-00069245.fits already exists!
Warninig: $OUTDIR/80619/spectra-7-80619-00069245.fits already exists!
Warninig: $OUTDIR/80619/spectra-8-80619-00069245.fits already exists!
Warninig: $OUTDIR/80619/spectra-9-80619-00069245.fits already exists!
Warninig: $OUTDIR/80619/spectra-0-80619-00069246.fits already exists!
Warninig: $OUTDIR/80

Warninig: $OUTDIR/80611/spectra-5-80611-00069410.fits already exists!
Warninig: $OUTDIR/80611/spectra-6-80611-00069410.fits already exists!
Warninig: $OUTDIR/80611/spectra-7-80611-00069410.fits already exists!
Warninig: $OUTDIR/80611/spectra-8-80611-00069410.fits already exists!
Warninig: $OUTDIR/80611/spectra-9-80611-00069410.fits already exists!
Warninig: $OUTDIR/80611/spectra-0-80611-00069411.fits already exists!
Warninig: $OUTDIR/80611/spectra-1-80611-00069411.fits already exists!
Warninig: $OUTDIR/80611/spectra-2-80611-00069411.fits already exists!
Warninig: $OUTDIR/80611/spectra-4-80611-00069411.fits already exists!
Warninig: $OUTDIR/80611/spectra-5-80611-00069411.fits already exists!
Warninig: $OUTDIR/80611/spectra-6-80611-00069411.fits already exists!
Warninig: $OUTDIR/80611/spectra-7-80611-00069411.fits already exists!
Warninig: $OUTDIR/80611/spectra-8-80611-00069411.fits already exists!
Warninig: $OUTDIR/80611/spectra-9-80611-00069411.fits already exists!
Warninig: $OUTDIR/80

Warninig: $OUTDIR/80620/spectra-7-80620-00069446.fits already exists!
Warninig: $OUTDIR/80620/spectra-8-80620-00069446.fits already exists!
Warninig: $OUTDIR/80620/spectra-9-80620-00069446.fits already exists!
Warninig: $OUTDIR/80620/spectra-0-80620-00069447.fits already exists!
Warninig: $OUTDIR/80620/spectra-1-80620-00069447.fits already exists!
Warninig: $OUTDIR/80620/spectra-2-80620-00069447.fits already exists!
Warninig: $OUTDIR/80620/spectra-4-80620-00069447.fits already exists!
Warninig: $OUTDIR/80620/spectra-5-80620-00069447.fits already exists!
Warninig: $OUTDIR/80620/spectra-7-80620-00069447.fits already exists!
Warninig: $OUTDIR/80620/spectra-8-80620-00069447.fits already exists!
Warninig: $OUTDIR/80620/spectra-9-80620-00069447.fits already exists!
Warninig: $OUTDIR/80614/spectra-0-80614-00069449.fits already exists!
Warninig: $OUTDIR/80614/spectra-1-80614-00069449.fits already exists!
Warninig: $OUTDIR/80614/spectra-2-80614-00069449.fits already exists!
Warninig: $OUTDIR/80

Warninig: $OUTDIR/80612/spectra-5-80612-00069597.fits already exists!
Warninig: $OUTDIR/80612/spectra-6-80612-00069597.fits already exists!
Warninig: $OUTDIR/80612/spectra-7-80612-00069597.fits already exists!
Warninig: $OUTDIR/80612/spectra-8-80612-00069597.fits already exists!
Warninig: $OUTDIR/80612/spectra-9-80612-00069597.fits already exists!
Warninig: $OUTDIR/80612/spectra-0-80612-00069599.fits already exists!
Warninig: $OUTDIR/80612/spectra-1-80612-00069599.fits already exists!
Warninig: $OUTDIR/80612/spectra-2-80612-00069599.fits already exists!
Warninig: $OUTDIR/80612/spectra-4-80612-00069599.fits already exists!
Warninig: $OUTDIR/80612/spectra-5-80612-00069599.fits already exists!
Warninig: $OUTDIR/80612/spectra-6-80612-00069599.fits already exists!
Warninig: $OUTDIR/80612/spectra-7-80612-00069599.fits already exists!
Warninig: $OUTDIR/80612/spectra-8-80612-00069599.fits already exists!
Warninig: $OUTDIR/80612/spectra-9-80612-00069599.fits already exists!
Warninig: $OUTDIR/80

## Print redrock commands

Include 15-minute timeout

In [9]:
command_output_path_all = 'rrdesi_all.txt'
command_output_path = 'rrdesi.txt'  # commands for reruns (e.g. after failures)

f_all = open(command_output_path_all, 'w')
f = open(command_output_path, 'w')

for expid in np.unique(cframes['expid']):

    mask = (cframes['expid']==expid)
    tileid = cframes['tileid'][mask][0]
    
    for petal_loc in np.unique(cframes['petal_loc'][mask]):
        
        # Stephen's suggested directory and filename format
        expid_str = str(expid).zfill(8)
        spectra_output_loc = os.path.join('$OUTDIR', str(tileid), 'spectra-{}-{}-{}.fits'.format(petal_loc, tileid, expid_str))

        redrock_output_loc = os.path.join('$OUTDIR', str(tileid), 'redrock-{}-{}-{}.h5'.format(petal_loc, tileid, expid_str))
        zbest_output_loc = os.path.join('$OUTDIR', str(tileid), 'zbest-{}-{}-{}.fits'.format(petal_loc, tileid, expid_str))
        log_loc = os.path.join('$OUTDIR', str(tileid), 'redrock-{}-{}-{}.log'.format(petal_loc, tileid, expid_str))

        f_all.write('srun -N 1 -n 32 -c 2 -t 00:15:00 rrdesi_mpi {} -o {} -z {} &> {}\n'.format(spectra_output_loc, redrock_output_loc, zbest_output_loc, log_loc))
        
        if os.path.isfile(os.path.expandvars(zbest_output_loc)) and (not overwrite):
            print('Warninig: {} already exists!'.format(zbest_output_loc))
        else:
            f.write('srun -N 1 -n 32 -c 2 -t 00:15:00 rrdesi_mpi {} -o {} -z {} &> {}\n'.format(spectra_output_loc, redrock_output_loc, zbest_output_loc, log_loc))
f.close()
f_all.close()

Warninig: $OUTDIR/70004/zbest-0-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/zbest-2-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/zbest-3-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/zbest-6-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/zbest-7-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/zbest-9-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/zbest-0-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/zbest-2-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/zbest-3-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/zbest-6-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/zbest-7-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/zbest-9-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/zbest-0-70004-00051001.fits already exists!
Warninig: $OUTDIR/70004/zbest-2-70004-00051001.fits already exists!
Warninig: $OUTDIR/70004/zbest-3-70004-00051001.f

Warninig: $OUTDIR/70508/zbest-3-70508-00052088.fits already exists!
Warninig: $OUTDIR/70508/zbest-6-70508-00052088.fits already exists!
Warninig: $OUTDIR/70508/zbest-7-70508-00052088.fits already exists!
Warninig: $OUTDIR/70508/zbest-9-70508-00052088.fits already exists!
Warninig: $OUTDIR/70508/zbest-0-70508-00052089.fits already exists!
Warninig: $OUTDIR/70508/zbest-3-70508-00052089.fits already exists!
Warninig: $OUTDIR/70508/zbest-6-70508-00052089.fits already exists!
Warninig: $OUTDIR/70508/zbest-7-70508-00052089.fits already exists!
Warninig: $OUTDIR/70508/zbest-9-70508-00052089.fits already exists!
Warninig: $OUTDIR/70502/zbest-0-70502-00052090.fits already exists!
Warninig: $OUTDIR/70502/zbest-3-70502-00052090.fits already exists!
Warninig: $OUTDIR/70502/zbest-6-70502-00052090.fits already exists!
Warninig: $OUTDIR/70502/zbest-7-70502-00052090.fits already exists!
Warninig: $OUTDIR/70502/zbest-9-70502-00052090.fits already exists!
Warninig: $OUTDIR/70502/zbest-0-70502-00052112.f

Warninig: $OUTDIR/70511/zbest-0-70511-00052925.fits already exists!
Warninig: $OUTDIR/70511/zbest-3-70511-00052925.fits already exists!
Warninig: $OUTDIR/70511/zbest-6-70511-00052925.fits already exists!
Warninig: $OUTDIR/70511/zbest-7-70511-00052925.fits already exists!
Warninig: $OUTDIR/70511/zbest-9-70511-00052925.fits already exists!
Warninig: $OUTDIR/70511/zbest-0-70511-00052926.fits already exists!
Warninig: $OUTDIR/70511/zbest-3-70511-00052926.fits already exists!
Warninig: $OUTDIR/70511/zbest-6-70511-00052926.fits already exists!
Warninig: $OUTDIR/70511/zbest-7-70511-00052926.fits already exists!
Warninig: $OUTDIR/70511/zbest-9-70511-00052926.fits already exists!
Warninig: $OUTDIR/70511/zbest-0-70511-00052927.fits already exists!
Warninig: $OUTDIR/70511/zbest-3-70511-00052927.fits already exists!
Warninig: $OUTDIR/70511/zbest-6-70511-00052927.fits already exists!
Warninig: $OUTDIR/70511/zbest-7-70511-00052927.fits already exists!
Warninig: $OUTDIR/70511/zbest-9-70511-00052927.f

Warninig: $OUTDIR/70510/zbest-9-70510-00053088.fits already exists!
Warninig: $OUTDIR/70510/zbest-0-70510-00053089.fits already exists!
Warninig: $OUTDIR/70510/zbest-3-70510-00053089.fits already exists!
Warninig: $OUTDIR/70510/zbest-6-70510-00053089.fits already exists!
Warninig: $OUTDIR/70510/zbest-7-70510-00053089.fits already exists!
Warninig: $OUTDIR/70510/zbest-9-70510-00053089.fits already exists!
Warninig: $OUTDIR/70510/zbest-0-70510-00053090.fits already exists!
Warninig: $OUTDIR/70510/zbest-3-70510-00053090.fits already exists!
Warninig: $OUTDIR/70510/zbest-6-70510-00053090.fits already exists!
Warninig: $OUTDIR/70510/zbest-7-70510-00053090.fits already exists!
Warninig: $OUTDIR/70510/zbest-9-70510-00053090.fits already exists!
Warninig: $OUTDIR/70003/zbest-0-70003-00053107.fits already exists!
Warninig: $OUTDIR/70003/zbest-3-70003-00053107.fits already exists!
Warninig: $OUTDIR/70003/zbest-6-70003-00053107.fits already exists!
Warninig: $OUTDIR/70003/zbest-7-70003-00053107.f

Warninig: $OUTDIR/66000/zbest-6-66000-00055445.fits already exists!
Warninig: $OUTDIR/66000/zbest-7-66000-00055445.fits already exists!
Warninig: $OUTDIR/66000/zbest-8-66000-00055445.fits already exists!
Warninig: $OUTDIR/66000/zbest-9-66000-00055445.fits already exists!
Warninig: $OUTDIR/65008/zbest-0-65008-00055456.fits already exists!
Warninig: $OUTDIR/65008/zbest-1-65008-00055456.fits already exists!
Warninig: $OUTDIR/65008/zbest-2-65008-00055456.fits already exists!
Warninig: $OUTDIR/65008/zbest-3-65008-00055456.fits already exists!
Warninig: $OUTDIR/65008/zbest-4-65008-00055456.fits already exists!
Warninig: $OUTDIR/65008/zbest-5-65008-00055456.fits already exists!
Warninig: $OUTDIR/65008/zbest-6-65008-00055456.fits already exists!
Warninig: $OUTDIR/65008/zbest-7-65008-00055456.fits already exists!
Warninig: $OUTDIR/65008/zbest-8-65008-00055456.fits already exists!
Warninig: $OUTDIR/65008/zbest-9-65008-00055456.fits already exists!
Warninig: $OUTDIR/65008/zbest-0-65008-00055457.f

...

Warninig: $OUTDIR/80618/zbest-4-80618-00069414.fits already exists!
Warninig: $OUTDIR/80618/zbest-5-80618-00069414.fits already exists!
Warninig: $OUTDIR/80618/zbest-6-80618-00069414.fits already exists!
Warninig: $OUTDIR/80618/zbest-7-80618-00069414.fits already exists!
Warninig: $OUTDIR/80618/zbest-8-80618-00069414.fits already exists!
Warninig: $OUTDIR/80618/zbest-9-80618-00069414.fits already exists!
Warninig: $OUTDIR/80618/zbest-0-80618-00069415.fits already exists!
Warninig: $OUTDIR/80618/zbest-1-80618-00069415.fits already exists!
Warninig: $OUTDIR/80618/zbest-2-80618-00069415.fits already exists!
Warninig: $OUTDIR/80618/zbest-4-80618-00069415.fits already exists!
Warninig: $OUTDIR/80618/zbest-5-80618-00069415.fits already exists!
Warninig: $OUTDIR/80618/zbest-6-80618-00069415.fits already exists!
Warninig: $OUTDIR/80618/zbest-7-80618-00069415.fits already exists!
Warninig: $OUTDIR/80618/zbest-8-80618-00069415.fits already exists!
Warninig: $OUTDIR/80618/zbest-9-80618-00069415.f

Warninig: $OUTDIR/80608/zbest-6-80608-00069438.fits already exists!
Warninig: $OUTDIR/80608/zbest-7-80608-00069438.fits already exists!
Warninig: $OUTDIR/80608/zbest-8-80608-00069438.fits already exists!
Warninig: $OUTDIR/80608/zbest-9-80608-00069438.fits already exists!
Warninig: $OUTDIR/80607/zbest-0-80607-00069440.fits already exists!
Warninig: $OUTDIR/80607/zbest-1-80607-00069440.fits already exists!
Warninig: $OUTDIR/80607/zbest-2-80607-00069440.fits already exists!
Warninig: $OUTDIR/80607/zbest-4-80607-00069440.fits already exists!
Warninig: $OUTDIR/80607/zbest-5-80607-00069440.fits already exists!
Warninig: $OUTDIR/80607/zbest-6-80607-00069440.fits already exists!
Warninig: $OUTDIR/80607/zbest-7-80607-00069440.fits already exists!
Warninig: $OUTDIR/80607/zbest-8-80607-00069440.fits already exists!
Warninig: $OUTDIR/80607/zbest-9-80607-00069440.fits already exists!
Warninig: $OUTDIR/80607/zbest-0-80607-00069441.fits already exists!
Warninig: $OUTDIR/80607/zbest-1-80607-00069441.f

Warninig: $OUTDIR/80616/zbest-2-80616-00069577.fits already exists!
Warninig: $OUTDIR/80616/zbest-4-80616-00069577.fits already exists!
Warninig: $OUTDIR/80616/zbest-5-80616-00069577.fits already exists!
Warninig: $OUTDIR/80616/zbest-6-80616-00069577.fits already exists!
Warninig: $OUTDIR/80616/zbest-7-80616-00069577.fits already exists!
Warninig: $OUTDIR/80616/zbest-8-80616-00069577.fits already exists!
Warninig: $OUTDIR/80616/zbest-9-80616-00069577.fits already exists!
Warninig: $OUTDIR/80616/zbest-0-80616-00069578.fits already exists!
Warninig: $OUTDIR/80616/zbest-1-80616-00069578.fits already exists!
Warninig: $OUTDIR/80616/zbest-2-80616-00069578.fits already exists!
Warninig: $OUTDIR/80616/zbest-4-80616-00069578.fits already exists!
Warninig: $OUTDIR/80616/zbest-5-80616-00069578.fits already exists!
Warninig: $OUTDIR/80616/zbest-6-80616-00069578.fits already exists!
Warninig: $OUTDIR/80616/zbest-7-80616-00069578.fits already exists!
Warninig: $OUTDIR/80616/zbest-8-80616-00069578.f

Warninig: $OUTDIR/80612/zbest-8-80612-00069596.fits already exists!
Warninig: $OUTDIR/80612/zbest-9-80612-00069596.fits already exists!
Warninig: $OUTDIR/80612/zbest-0-80612-00069597.fits already exists!
Warninig: $OUTDIR/80612/zbest-1-80612-00069597.fits already exists!
Warninig: $OUTDIR/80612/zbest-2-80612-00069597.fits already exists!
Warninig: $OUTDIR/80612/zbest-4-80612-00069597.fits already exists!
Warninig: $OUTDIR/80612/zbest-5-80612-00069597.fits already exists!
Warninig: $OUTDIR/80612/zbest-6-80612-00069597.fits already exists!
Warninig: $OUTDIR/80612/zbest-7-80612-00069597.fits already exists!
Warninig: $OUTDIR/80612/zbest-8-80612-00069597.fits already exists!
Warninig: $OUTDIR/80612/zbest-9-80612-00069597.fits already exists!
Warninig: $OUTDIR/80612/zbest-0-80612-00069599.fits already exists!
Warninig: $OUTDIR/80612/zbest-1-80612-00069599.fits already exists!
Warninig: $OUTDIR/80612/zbest-2-80612-00069599.fits already exists!
Warninig: $OUTDIR/80612/zbest-4-80612-00069599.f

Warninig: $OUTDIR/80622/zbest-5-80622-00069632.fits already exists!
Warninig: $OUTDIR/80622/zbest-6-80622-00069632.fits already exists!
Warninig: $OUTDIR/80622/zbest-7-80622-00069632.fits already exists!
Warninig: $OUTDIR/80622/zbest-8-80622-00069632.fits already exists!
Warninig: $OUTDIR/80622/zbest-9-80622-00069632.fits already exists!


__Split the jobs into two batches for 40-node runs__

In [10]:
cframes_all = cframes.copy()

In [11]:
command_output_path = 'rrdesi_1.txt'  # commands for reruns (e.g. after failures)
cframes = cframes_all[:len(cframes_all)//2]
print(len(cframes))

f = open(command_output_path, 'w')

for expid in np.unique(cframes['expid']):

    mask = (cframes['expid']==expid)
    tileid = cframes['tileid'][mask][0]
    
    for petal_loc in np.unique(cframes['petal_loc'][mask]):
        
        # Stephen's suggested directory and filename format
        expid_str = str(expid).zfill(8)
        spectra_output_loc = os.path.join('$OUTDIR', str(tileid), 'spectra-{}-{}-{}.fits'.format(petal_loc, tileid, expid_str))

        redrock_output_loc = os.path.join('$OUTDIR', str(tileid), 'redrock-{}-{}-{}.h5'.format(petal_loc, tileid, expid_str))
        zbest_output_loc = os.path.join('$OUTDIR', str(tileid), 'zbest-{}-{}-{}.fits'.format(petal_loc, tileid, expid_str))
        log_loc = os.path.join('$OUTDIR', str(tileid), 'redrock-{}-{}-{}.log'.format(petal_loc, tileid, expid_str))

        if os.path.isfile(os.path.expandvars(zbest_output_loc)) and (not overwrite):
            print('Warninig: {} already exists!'.format(zbest_output_loc))
        else:
            f.write('srun -N 1 -n 32 -c 2 -t 00:10:00 rrdesi_mpi {} -o {} -z {} &> {}\n'.format(spectra_output_loc, redrock_output_loc, zbest_output_loc, log_loc))
f.close()

1926
Warninig: $OUTDIR/70004/zbest-0-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/zbest-2-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/zbest-6-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/zbest-7-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/zbest-9-70004-00050988.fits already exists!
Warninig: $OUTDIR/70004/zbest-0-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/zbest-2-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/zbest-3-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/zbest-6-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/zbest-7-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/zbest-9-70004-00050995.fits already exists!
Warninig: $OUTDIR/70004/zbest-0-70004-00051001.fits already exists!
Warninig: $OUTDIR/70004/zbest-2-70004-00051001.fits already exists!
Warninig: $OUTDIR/70004/zbest-3-70004-00051001.fits already exists!
Warninig: $OUTDIR/70004/zbest-7-70004-00051

Warninig: $OUTDIR/70508/zbest-9-70508-00052087.fits already exists!
Warninig: $OUTDIR/70508/zbest-0-70508-00052088.fits already exists!
Warninig: $OUTDIR/70508/zbest-3-70508-00052088.fits already exists!
Warninig: $OUTDIR/70508/zbest-6-70508-00052088.fits already exists!
Warninig: $OUTDIR/70508/zbest-7-70508-00052088.fits already exists!
Warninig: $OUTDIR/70508/zbest-9-70508-00052088.fits already exists!
Warninig: $OUTDIR/70508/zbest-0-70508-00052089.fits already exists!
Warninig: $OUTDIR/70508/zbest-6-70508-00052089.fits already exists!
Warninig: $OUTDIR/70508/zbest-7-70508-00052089.fits already exists!
Warninig: $OUTDIR/70508/zbest-9-70508-00052089.fits already exists!
Warninig: $OUTDIR/70502/zbest-0-70502-00052090.fits already exists!
Warninig: $OUTDIR/70502/zbest-6-70502-00052090.fits already exists!
Warninig: $OUTDIR/70502/zbest-7-70502-00052090.fits already exists!
Warninig: $OUTDIR/70502/zbest-9-70502-00052090.fits already exists!
Warninig: $OUTDIR/70502/zbest-0-70502-00052112.f

Warninig: $OUTDIR/68001/zbest-3-68001-00055613.fits already exists!
Warninig: $OUTDIR/68001/zbest-4-68001-00055613.fits already exists!
Warninig: $OUTDIR/68001/zbest-5-68001-00055613.fits already exists!
Warninig: $OUTDIR/68001/zbest-6-68001-00055613.fits already exists!
Warninig: $OUTDIR/68001/zbest-7-68001-00055613.fits already exists!
Warninig: $OUTDIR/68001/zbest-8-68001-00055613.fits already exists!
Warninig: $OUTDIR/68001/zbest-9-68001-00055613.fits already exists!
Warninig: $OUTDIR/68002/zbest-0-68002-00055626.fits already exists!
Warninig: $OUTDIR/68002/zbest-1-68002-00055626.fits already exists!
Warninig: $OUTDIR/68002/zbest-2-68002-00055626.fits already exists!
Warninig: $OUTDIR/68002/zbest-3-68002-00055626.fits already exists!
Warninig: $OUTDIR/68002/zbest-4-68002-00055626.fits already exists!
Warninig: $OUTDIR/68002/zbest-5-68002-00055626.fits already exists!
Warninig: $OUTDIR/68002/zbest-6-68002-00055626.fits already exists!
Warninig: $OUTDIR/68002/zbest-7-68002-00055626.f

Warninig: $OUTDIR/80605/zbest-1-80605-00067712.fits already exists!
Warninig: $OUTDIR/80605/zbest-2-80605-00067712.fits already exists!
Warninig: $OUTDIR/80605/zbest-3-80605-00067712.fits already exists!
Warninig: $OUTDIR/80605/zbest-4-80605-00067712.fits already exists!
Warninig: $OUTDIR/80605/zbest-5-80605-00067712.fits already exists!
Warninig: $OUTDIR/80605/zbest-6-80605-00067712.fits already exists!
Warninig: $OUTDIR/80605/zbest-7-80605-00067712.fits already exists!
Warninig: $OUTDIR/80605/zbest-8-80605-00067712.fits already exists!
Warninig: $OUTDIR/80605/zbest-9-80605-00067712.fits already exists!
Warninig: $OUTDIR/80605/zbest-0-80605-00067713.fits already exists!
Warninig: $OUTDIR/80605/zbest-1-80605-00067713.fits already exists!
Warninig: $OUTDIR/80605/zbest-2-80605-00067713.fits already exists!
Warninig: $OUTDIR/80605/zbest-3-80605-00067713.fits already exists!
Warninig: $OUTDIR/80605/zbest-4-80605-00067713.fits already exists!
Warninig: $OUTDIR/80605/zbest-5-80605-00067713.f

Warninig: $OUTDIR/80619/zbest-7-80619-00068673.fits already exists!
Warninig: $OUTDIR/80619/zbest-8-80619-00068673.fits already exists!
Warninig: $OUTDIR/80619/zbest-9-80619-00068673.fits already exists!
Warninig: $OUTDIR/80619/zbest-0-80619-00068674.fits already exists!


In [12]:
command_output_path = 'rrdesi_2.txt'  # commands for reruns (e.g. after failures)
cframes = cframes_all[len(cframes_all)//2:]
print(len(cframes))

f = open(command_output_path, 'w')

for expid in np.unique(cframes['expid']):

    mask = (cframes['expid']==expid)
    tileid = cframes['tileid'][mask][0]
    
    for petal_loc in np.unique(cframes['petal_loc'][mask]):
        
        # Stephen's suggested directory and filename format
        expid_str = str(expid).zfill(8)
        spectra_output_loc = os.path.join('$OUTDIR', str(tileid), 'spectra-{}-{}-{}.fits'.format(petal_loc, tileid, expid_str))

        redrock_output_loc = os.path.join('$OUTDIR', str(tileid), 'redrock-{}-{}-{}.h5'.format(petal_loc, tileid, expid_str))
        zbest_output_loc = os.path.join('$OUTDIR', str(tileid), 'zbest-{}-{}-{}.fits'.format(petal_loc, tileid, expid_str))
        log_loc = os.path.join('$OUTDIR', str(tileid), 'redrock-{}-{}-{}.log'.format(petal_loc, tileid, expid_str))

        if os.path.isfile(os.path.expandvars(zbest_output_loc)) and (not overwrite):
            print('Warninig: {} already exists!'.format(zbest_output_loc))
        else:
            f.write('srun -N 1 -n 32 -c 2 -t 00:10:00 rrdesi_mpi {} -o {} -z {} &> {}\n'.format(spectra_output_loc, redrock_output_loc, zbest_output_loc, log_loc))
f.close()

1927
Warninig: $OUTDIR/80619/zbest-1-80619-00068674.fits already exists!
Warninig: $OUTDIR/80619/zbest-2-80619-00068674.fits already exists!
Warninig: $OUTDIR/80619/zbest-4-80619-00068674.fits already exists!
Warninig: $OUTDIR/80619/zbest-5-80619-00068674.fits already exists!
Warninig: $OUTDIR/80619/zbest-6-80619-00068674.fits already exists!
Warninig: $OUTDIR/80619/zbest-7-80619-00068674.fits already exists!
Warninig: $OUTDIR/80619/zbest-8-80619-00068674.fits already exists!
Warninig: $OUTDIR/80619/zbest-9-80619-00068674.fits already exists!
Warninig: $OUTDIR/80621/zbest-0-80621-00068678.fits already exists!
Warninig: $OUTDIR/80621/zbest-1-80621-00068678.fits already exists!
Warninig: $OUTDIR/80621/zbest-2-80621-00068678.fits already exists!
Warninig: $OUTDIR/80621/zbest-4-80621-00068678.fits already exists!
Warninig: $OUTDIR/80621/zbest-5-80621-00068678.fits already exists!
Warninig: $OUTDIR/80621/zbest-6-80621-00068678.fits already exists!
Warninig: $OUTDIR/80621/zbest-7-80621-00068